<div style="color:red;font-weight:bold;background:yellow;text-align:center;padding:10px;border:solid">
    <h1>RUN IN EMR CLUSTER ONLY</h1>
    If the URL of the current page does not begin with "ec2", then do **NOT** proceed!
</div>

# SparkSQL Practice
In this practice, you will use the tools you learned in the readings and labs to perform some dataset querying.

## Connecting to PySpark

In [1]:
name = !hostname
if "dsa" in name[0]:
    raise RuntimeError("Only run this notebook in the EMR Cluster!")
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("pyspark-lab")
sc = SparkContext(conf=conf)

## 1
Load the Datset 

For this part, we will use the Titanic Dataset, located at `/datasets/titanic.csv`. Ingest this dataset in the cell below

In [2]:
from pyspark.sql import SQLContext

# To use Spark SQL we create a SQLContext from SparkContext
sqlContext = SQLContext(sc)

# Location of the dataset on HDFS
DATASET = '/datasets/titanic.csv'

df = sqlContext.read.format('com.databricks.spark.csv').options(
    header='true', inferschema='true').load(DATASET)

In [3]:
df.head()

Row(pclass=3, sex=1, age=38.0, sibsp=0, parch=0, fare=7.8958, embarked=1, survived=0)

In [4]:
df.createOrReplaceTempView("database")

## 2
What was the average age?

In [5]:
query = "SELECT AVG(age) FROM database"
sqlContext.sql(query).show()

+------------------+
|          avg(age)|
+------------------+
|29.548696629213484|
+------------------+



## 3
Show the average `fare` per `pclass` and order it by the average `fare` from highest to lowest

In [6]:
query = "SELECT pclass,AVG(fare) as average FROM database GROUP BY pclass ORDER BY average DESC"

sqlContext.sql(query).show()

+------+------------------+
|pclass|           average|
+------+------------------+
|     1| 88.19726064814813|
|     2|20.172962222222225|
|     3|13.297156477732804|
+------+------------------+



## 4
What is the average fare for men (sex = 1) in their 20s?

In [7]:
query = "SELECT AVG(fare) as average FROM database where (age < 30 AND age >19) AND sex =1 "

sqlContext.sql(query).show()

+------------------+
|           average|
+------------------+
|17.192192063492058|
+------------------+



## 5
What is the ratio of men to women per class?

In [8]:
query = """
SELECT pclass,sum(case when sex = 1 then 1 else 0 end)/ sum(case when sex = 0 then 1 else 0 end) as ratio
FROM database 
GROUP BY pclass
ORDER BY pclass
"""

sqlContext.sql(query).show()


+------+------------------+
|pclass|             ratio|
+------+------------------+
|     1|1.3225806451612903|
|     2|1.5714285714285714|
|     3|2.1870967741935483|
+------+------------------+



## 6
What percentage of women (sex = 0) survived in class 3?

Hint: You may need 2 separate queries. If your query returns one number, you can fetch it by turning the SQL result into a pandas dataframe and then using `df.iloc[0,0]`

In [9]:
query1 = "SELECT count(*) FROM database WHERE sex = 0 AND pclass=3"
query2 = "SELECT count(*) FROM database WHERE sex = 0 AND pclass=3 AND survived =1"

df = sqlContext.sql(query1).toPandas()
total = df.iloc[0,0]

df = sqlContext.sql(query2).toPandas()
survived = df.iloc[0,0]

pct = survived / total * 100
print("{:.2f}%".format(pct))

50.97%


## 7
What is the average age per class?

In [10]:
query = """
SELECT pclass,avg(age)
FROM database 
GROUP BY pclass
ORDER BY pclass
"""

sqlContext.sql(query).show()


+------+------------------+
|pclass|          avg(age)|
+------+------------------+
|     1|37.988055555555555|
|     2|29.550944444444443|
|     3|25.857793522267208|
+------+------------------+

